# PixelPrism Math Basics

This notebook walks through the foundational math containers used across PixelPrism:
- `DType` captures scalar/element types.
- `Shape` stores symbolic tensor dimensions.
- `Value` ties user data with `Shape`/`DType` metadata.
Each section below demonstrates typical usage patterns and key behaviors.

In [ ]:
from pixelprism.math.dtype import DType
from pixelprism.math.shape import Shape
from pixelprism.math.value import Value

## DType

In [ ]:
floats = (DType.FLOAT32, DType.FLOAT64)
ints = (DType.INT32, DType.INT64)
summary = []
for dt in DType:
    summary.append({
        'name': dt.name,
        'value': dt.value,
        'is_float': dt.is_float,
        'is_int': dt.is_int,
        'is_bool': dt.is_bool
    })
summary

The helper method `DType.promote(a, b)` mimics how binary ops pick a resulting dtype. 
The ordering favors more expressive types, so mixing `FLOAT32` with `INT32` yields `FLOAT32`.

In [ ]:
DType.promote(DType.FLOAT32, DType.INT32)

## Shape

In [ ]:
vector = Shape.vector(3)
matrix = Shape.matrix(2, 4)
unknown = Shape((None, 4))
{'vector': vector, 'matrix': matrix, 'unknown': unknown, 'matrix_rank': matrix.rank, 'unknown_size': unknown.size}

Shapes can verify broadcast/elementwise compatibility and produce derived shapes for ops like `matmul`.

In [ ]:
lhs = Shape((2, 3, 4))
rhs = Shape((2, 4, 5))
result = lhs.matmul_result(rhs)
compat = lhs.merge_elementwise(lhs)
{'matmul': result, 'elementwise': compat, 'can_reshape': lhs.can_reshape(Shape((4, 3, 4)))}

## Value

In [ ]:
data = [[1.0, 2.0], [3.0, 4.0]]
value = Value(data=data, shape=Shape.matrix(2, 2), dtype=DType.FLOAT32)
{'shape': value.shape, 'dtype': value.dtype, 'mutable': value.mutable, 'data': value.get()}

The value enforces the declared shape when wrapping nested Python data. Attempting to assign data
with mismatched dimensions raises `ValueError`, while creating an immutable value prevents future
updates.

In [ ]:
immutable = Value(data=[1, 2, 3], shape=Shape.vector(3), dtype=DType.INT32, mutable=False)
try:
    immutable.set([4, 5, 6])
except RuntimeError as exc:
    exc